In [1]:
# Import packages
import os

import glob
import warnings

import numpy as np
import astropy.units as u
from astropy.io import fits
from astropy.table import Table, join


# Import grizli
import grizli
from grizli import multifit, fitting
from grizli.pipeline import auto_script
import grizli.pipeline.photoz
import pickle
fname='gru-00' # Field name

In [2]:
grism_files = glob.glob('%s/Prep/*GrismFLT.fits'  % fname) # Modify this to be the directory where *GrismFLT.fits are 

In [ ]:
# read grism images of the entire field
grp = multifit.GroupFLT(
        grism_files=grism_files,
        catalog='%s/Prep/%s-ir.cat.fits' % (fname,fname), # Modify this to be the location where the catalog is
        cpu_count=8,
        sci_extn=1,
        pad=800,
    )

In [ ]:
#object_id we want to extract the beams from *.ir.cat.fits
# This will make the file *beams.fits for that source
# 
beams = grp.get_beams(object_id, size=32, min_mask=0, min_sens=0.0)
if len(beams) > 0:
    # Extract beam
    mb = multifit.MultiBeam(
        beams, fcontam=1, min_sens=0.0, min_mask=0, group_name=fname
    )
    mb.write_master_fits() 


In [ ]:
# Generate the defaul fit parameters. The script will generate the fit_args.npy file
pline = {
        'kernel': 'square',
        'pixfrac': 0.5,
        'pixscale': 0.04,
        'size': 8,
        'wcs': None,
    }
args=auto_script.generate_fit_params(
       fit_only_beams=True,
       fit_beams=True,
       run_fit=True,
       poly_order=7,
       sys_err=0.1,
       fcontam=1,
       zr=[0.1,12],
       fit_trace_shift=True,
       save_file='fit_args.npy',
   pline=pline, field_root=fname, min_sens=0.0, min_mask=0.0
)

In [ ]:
# Run this code inside the directory where the above *beams.fits is saved, and where fit_args.npy, *phot_apcorr.fits are exist.

# Generate the internal NIRISS photometry for Grizli redshift fitting
args = np.load('fit_args.npy', allow_pickle=True)[0]
aper_ix = 1
total_flux_column = 'flux'
get_external_photometry = False
int_ez = grizli.pipeline.photoz.eazy_photoz(fname, force=False, object_only=True,
              apply_background=True, aper_ix=aper_ix,
              apply_prior=True, beta_prior=True,
              get_external_photometry=get_external_photometry,
              external_limits=3, external_sys_err=0.3, external_timeout=300,
              sys_err=args['sys_err'], z_step=0.01, z_min=0.01, z_max=12,
              total_flux=total_flux_column)
t0 = args['t0']
int_phot_obj = grizli.pipeline.photoz.EazyPhot(int_ez, grizli_templates=t0, zgrid=int_ez.zgrid)
# Save as a pickle to use later for other sources.

with open('outthere-niriss-%s_eazyphot.pkl' % (fname), "wb") as file:  # Open the file in binary write mode
    pickle.dump(int_phot_obj, file)

# if len(glob.glob('outthere-niriss-%s_eazyphot.pkl' % (fname)))==1:
#     with open('outthere-niriss-%s_eazyphot.pkl' % (fname),'rb') as f:
#         int_phot_obj = pickle.load(f)

In [ ]:
#Now generate the fit parameter file again using the NIRISS internal photometry 
order=0
args_withPhot=auto_script.generate_fit_params(
       fit_only_beams=True,
       fit_beams=True,
       run_fit=True,
       poly_order=7,
       sys_err=0.1,
       fcontam=1,
       zr=[0.1,12],
       fit_trace_shift=True,
       use_phot_obj=True,
       scale_photometry=order+1,
       phot_obj=int_phot_obj, # This is photometric object we created above
       save_file='fit_args_wNIRISSphot.npy',
       pline=pline, field_root=fname, min_sens=0.00, min_mask=0.0
    )

In [ ]:
args_withPhot = np.load('fit_args_wNIRISSphot.npy', allow_pickle=True)[0]

In [ ]:
# Now run the redshift fitting for a given object_id
fitting.run_all_parallel(object_id, args_file='fit_args_wNIRISSphot.npy', 
                         verbose=True,fit_only_beams=False, only_stacks= False,fit_beams=True)
